In [23]:
import re, json, time
from selenium import webdriver

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)

    selector = Selector(driver.page_source)
    driver.quit()

    return selector



In [24]:
# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

print(canais)


['@LuanMorenoMMaciel', '@GrupoGPSSA', '@DarshilParmar', '@TRRaveendra', '@Akitando']


In [25]:
base_url = "https://socialblade.com/youtube/channel/"

social_blade_data = []
for c in canais:
    response = scroll_page(base_url + c)

    # last 30 days views #socialblade-user-content > div:nth-child(23) > div:nth-child(3) > span
    views = response.css('#socialblade-user-content > div:nth-child(23) > div:nth-child(3) > span::text').get()
    subs = response.css("#socialblade-user-content > div:nth-child(23) > div:nth-child(2) > span::text").get()

    if views is None or subs is None:
        views = response.css('#socialblade-user-content > div:nth-child(16) > div:nth-child(3) > span::text').get()
        subs = response.css("#socialblade-user-content > div:nth-child(16) > div:nth-child(2) > span::text").get()



    social_blade_data.append({"Channel": c, "views": views, "subs": subs})


In [26]:
last_30days = pd.DataFrame(social_blade_data)

#socialblade-user-content > div:nth-child(16) > div:nth-child(3) > span

#socialblade-user-content > div:nth-child(16) > div:nth-child(2) > span

In [27]:
last_30days = last_30days.fillna('0')

In [28]:
last_30days

,Channel,views,subs
0,@LuanMorenoMMaciel,"+16,239",+270
1,@GrupoGPSSA,"+79,812",+1.3K
2,@DarshilParmar,"+228,744",+4.4K
3,@TRRaveendra,"+99,203",+1.2K
4,@Akitando,"+573,536",+12K


In [29]:
def fix_views(view: pd.Series) -> pd.Series:

    view = view.str.replace("+", "").str.replace(",", "")
    # print(view)

    view = view.apply(lambda x: float(x.replace("K", "")) * 1000 if 'K' in x else x)
    view = view.apply(lambda x: float(x.replace("M", "")) * 1000000 if 'M' in str(x) else x)

    return view

last_30days = last_30days.assign(
    views=lambda x: fix_views(x["views"]),
    subs= lambda x: fix_views(x["subs"])
)

In [30]:
# create folder for lead

import os

os.makedirs(f"data/{canais[0].split('@')[-1]}", exist_ok=True)


In [31]:


last_30days.to_csv(f'data/{canais[0].split("@")[-1]}/last_30days.csv', index=False)

In [32]:
last_30days

,Channel,views,subs
0,@LuanMorenoMMaciel,16239,270
1,@GrupoGPSSA,79812,1300.0
2,@DarshilParmar,228744,4400.0
3,@TRRaveendra,99203,1200.0
4,@Akitando,573536,12000.0
